# Business Results

## Recomendação de compra dos imóveis:
##### Para decidir quais imóveis deverão ser comprados, iremos comparar os imóveis pelo zipcode e selecionar as casas que estão abaixo da média. Como explicado anteriormente, estaremos selecionando as casas que possuem 'condition' maior ou igual a 3.

In [32]:
import pandas as pd

data = pd.read_csv('datasets/kc_house_clean.csv')
pd.set_option( 'display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [33]:
#comparar a média do 'zipcode' e se o valor for abaixo da média e a condição >= 3 então comprar
df = data[['price','id','zipcode','condition']].copy()

recomendations = df[['zipcode','price']].groupby('zipcode').median().reset_index()
recomendations.columns = ['zipcode','median_price']
df = pd.merge(df, recomendations, on= 'zipcode', how = 'inner')

df['recomendations'] = 'na'
for i in range(len(df)):
    if (df.loc[i,'price'] < df.loc[i,'median_price'])&(df.loc[i,'condition'] >= 3):
        df.loc[i,'recomendations'] = 'comprar'
    else:
        df.loc[i,'recomendations'] = 'não comprar'
df.to_csv('datasets/recomendacoes_compras.csv')

In [34]:
df.head()

,price,id,zipcode,condition,median_price,recomendations
0,221900.00,7129300520,98178,3,278277.00,comprar
1,205425.00,4060000240,98178,4,278277.00,comprar
2,445000.00,4058801670,98178,3,278277.00,não comprar
3,236000.00,2976800796,98178,3,278277.00,comprar
4,170000.00,6874200960,98178,3,278277.00,comprar


In [39]:
## Recomendação de venda dos imóveis
##### Para decidir o melhor momento de venda dos imóveis estaremos comparando os imóveis pelo 'zipcode' e pela estação.
##### Se o preço for menor que a mediana, então estaremos acrescentando 30% ao valor.
##### Se o preço for maior que a mediana, então estaremos acrescentando 10% ao valor.

data = pd.read_csv('datasets/kc_house_clean.csv')
df1 = data[['price','date','zipcode','id']].copy()

df1['date'] = pd.to_datetime(df1['date']).dt.month
df1['season'] = df1['date'].apply(lambda x: 'spring' if ( x >= 3 )&( x <= 5 ) else
                                              'summer' if ( x >= 6 )&( x <= 8 ) else
                                              'fall' if ( x >= 9 )&( x <= 11 ) else 
                                              'winter')


estacoes = df1[['zipcode','season','price']].groupby(['zipcode','season']).median().reset_index()
estacoes.columns = ['zipcode','season','median_price']
estacoes['zip_season'] = estacoes['zipcode'].astype(str) + "_" + estacoes['season'].astype(str)
estacoes = estacoes.drop(['zipcode','season'], axis = 1)

df1['zip_season'] = df1['zipcode'].astype(str) + "_" + df1['season'].astype(str)
df1 = pd.merge( df1, estacoes, on='zip_season', how='inner')

df1['venda'] = 'na'
for i in range(len(data)):
    if (df1.loc[i,'price'] <= df1.loc[i,'median_price']):
        df1.loc[i,'venda'] = df1.loc[i,'price'] * 1.30
    else:
        df1.loc[i,'venda'] = df1.loc[i,'price'] * 1.10
df1.to_csv('datasets/recomendacoes_venda.csv')

In [40]:
df1.head()

,price,date,zipcode,id,season,zip_season,median_price,venda
0,221900.00,10,98178,7129300520,fall,98178_fall,290500.00,288470.00
1,236000.00,9,98178,2976800796,fall,98178_fall,290500.00,306800.00
2,190000.00,9,98178,1180003090,fall,98178_fall,290500.00,247000.00
3,350000.00,9,98178,2171400197,fall,98178_fall,290500.00,385000.00
4,299000.00,9,98178,1180002378,fall,98178_fall,290500.00,328900.00
